In [1]:
# %load main.py
import sys
# import importlib

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import datetime as dt

from armax import armax
from config import config
import mysql_utils as mysql
import visualization

In [2]:
DATA_FREQUENCY = dt.timedelta(minutes=5)

In [3]:
#DETECTOR_DATA_TABLE = "detector_data_processed_2017_1"
DETECTOR_DATA_TABLE = "detector_data_processed_2017"
DETECTOR_ID = "608219"
DETECTOR_DATA_QUERY = "SELECT DetectorID, Year, Month, Day, Time, Volume, Occupancy\
                        FROM {} AS DD NATURAL JOIN detector_health AS DH\
                        WHERE DetectorID = {} AND Health = 1\
                        ORDER BY Year, Month, Day, Time;".format(DETECTOR_DATA_TABLE, DETECTOR_ID)

In [4]:
def query_detector_data(cursor, table, detector_id, graph=False):
    query = DETECTOR_DATA_QUERY.format(table, detector_id)

    cursor = mysql.query(cursor, query)
    
    if cursor == None:
        return

    time = []
    volume = []
    occupancy = []
    #speed = []

    for row in cursor:
        d = dt.datetime(row[1], row[2], row[3], row[4] // 3600, (row[4] % 3600) // 60, row[4] % 60)
        time.append(d)

        volume.append(row[5])
        occupancy.append(row[6])
        #speed.append(row[7])

    time = np.array(time)
    volume = np.array(volume)
    occupancy = np.array(occupancy)
    occupancy_percentage = occupancy / 3600 * 100
    #speed = np.array(speed)

    if graph:
        visualization.plot_data_over_time(time, volume, title="Detector {} Volume 2017".format(detector_id), ylabel="Volume (vph)", figsize=(12, 5))
        visualization.plot_data_over_time(time, occupancy, title="Detector {} Occupancy 2017".format(detector_id), ylabel="Occupancy (s)", figsize=(12, 5))
        #visualization.plot_data_over_time(time, speed, title="Detector {} Speed 2017".format(detector_id), ylabel="Speed", figsize=(12, 5))
        visualization.plot_data_over_time(time, occupancy_percentage, title="Detector {} Occupancy 2017".format(detector_id), ylabel="Occupancy (%)", figsize=(12, 5))
        visualization.plot_fundamental_diagram(volume, occupancy_percentage, title="Detector {} Flow-Occupancy Diagram 2017".format(detector_id))

    return time, volume, occupancy

In [5]:
cnx = mysql.connect_to_database(**config)

if cnx == None:
    sys.exit()

cursor = cnx.cursor()

In [6]:
time, flow, occupancy = query_detector_data(cursor, DETECTOR_DATA_TABLE, DETECTOR_ID, graph=False)

In [7]:
flow = pd.DataFrame(flow, index=time).reindex(pd.date_range(min(time), max(time), freq='5T'), fill_value=np.nan)
flow = flow.interpolate(method='linear')

In [8]:
armax_model = armax(flow, dates=time, frequency=DATA_FREQUENCY)

In [ ]:
armax_model.fit_to_order((1, 1), method="css", cross_validate=True, verbose=True)

Fitting order (1, 1)
Cross validating monthly results in 11 folds
Trained month 0; sse 399082816.7072193


In [ ]:
armax_model.fit(method="css", cross_validate=True, verbose=True)

In [ ]:
for order in armax_model._armax_models:
    m = armax_model._armax_models[order]
    print(order)
    print(m.aic)
    print(m.bic)
    print(np.sum(np.power(m.resid, 2)))

In [ ]:
print(armax_model.best_model_order)

In [ ]:
a = armax_model._armax_models[(3, 3)]

In [ ]:
b = armax_model._armax_models[(1, 1)]

In [ ]:
index0 = min(range(len(time)), key=lambda i: (1 - (time[i].month == 9)))
index1 = min(range(len(time)), key=lambda i: (1 - (time[i].month == 10)))
index2 = min(range(len(time)), key=lambda i: (1 - (time[i].month == 11)))
index3 = min(range(len(time)), key=lambda i: (1 - (time[i].month == 12)))

In [ ]:
fig, ax = plt.subplots(figsize=(20, 5))
fig.autofmt_xdate()
plt.plot(time[index0:index1], flow[index0:index1])
plt.plot(time[index0:index1], a.fittedvalues[index0:index1])
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(20, 5))
fig.autofmt_xdate()
plt.plot(time[index1:index2], flow[index1:index2])
plt.plot(time[index1:index2], a.fittedvalues[index1:index2])
plt.show()

In [ ]:
cursor.close()
cnx.close()